In [1]:
from netunicorn.base import Experiment, Task, Pipeline
from netunicorn.client.remote import RemoteClient

In [2]:
from typing import Literal

from netunicorn.base import Task
from netunicorn.base.types import FlagValues


class SetFlagTask(Task):
    def __init__(self, flag_name: str, flag_values: FlagValues, *args, **kwargs):
        if flag_values.int_value is None and flag_values.text_value is None:
            raise ValueError("Either int_value or text_value must be set")

        self.flag_name = flag_name
        self.flag_values = flag_values
        super().__init__(*args, **kwargs)

    def run(self) -> None:
        import requests as req
        import os

        gateway = os.environ["NETUNICORN_GATEWAY_ENDPOINT"]
        experiment_id = os.environ["NETUNICORN_EXPERIMENT_ID"]

        req.post(
            f"{gateway}/api/v1/experiment/{experiment_id}/flag/{self.flag_name}",
            json=self.flag_values.dict(),
        ).raise_for_status()


class GetFlagTask(Task):
    def __init__(self, flag_name: str, *args, **kwargs):
        self.flag_name = flag_name
        super().__init__(*args, **kwargs)

    def run(self) -> FlagValues:
        import requests as req
        import os

        gateway = os.environ["NETUNICORN_GATEWAY_ENDPOINT"]
        experiment_id = os.environ["NETUNICORN_EXPERIMENT_ID"]

        result = req.get(
            f"{gateway}/api/v1/experiment/{experiment_id}/flag/{self.flag_name}"
        ).json()
        return FlagValues(**result)


class _AtomicOperationFlagTask(Task):
    def __init__(self, flag_name: str, operation: Literal['increment', 'decrement'], *args, **kwargs):
        self.flag_name = flag_name
        self.operation = operation
        super().__init__(*args, **kwargs)

    def run(self) -> None:
        import requests as req
        import os

        gateway = os.environ["NETUNICORN_GATEWAY_ENDPOINT"]
        experiment_id = os.environ["NETUNICORN_EXPERIMENT_ID"]

        req.post(
            f"{gateway}/api/v1/experiment/{experiment_id}/flag/{self.flag_name}/{self.operation}",
        ).raise_for_status()


class AtomicIncrementFlagTask(_AtomicOperationFlagTask):
    def __init__(self, flag_name: str, *args, **kwargs):
        super().__init__(flag_name, 'increment', *args, **kwargs)


class AtomicDecrementFlagTask(_AtomicOperationFlagTask):
    def __init__(self, flag_name: str, *args, **kwargs):
        super().__init__(flag_name, 'decrement', *args, **kwargs)


In [3]:
experiment_name = 'my_cool_experiment'
flag_name = 'test'

In [4]:
pipeline = Pipeline()
pipeline.environment_definition.image = "localhost:5000/netunicorn-executor:latest"
(
    pipeline
    .then(
        SetFlagTask(experiment_name, FlagValues(text_value='abcd')),
    )
    .then([
        AtomicIncrementFlagTask(flag_name),
        AtomicIncrementFlagTask(flag_name)
    ])
    .then([
        AtomicDecrementFlagTask(flag_name),
        AtomicDecrementFlagTask(flag_name),
        AtomicDecrementFlagTask(flag_name),
        AtomicDecrementFlagTask(flag_name),
        AtomicDecrementFlagTask(flag_name),
        AtomicDecrementFlagTask(flag_name),
        AtomicDecrementFlagTask(flag_name),
        AtomicDecrementFlagTask(flag_name),
    ])
    .then(
        GetFlagTask(flag_name)
    )
)

Pipeline(d378e2cd-bb65-4100-9312-127e9b9d79bd): [[<__main__.SetFlagTask with name c27b05f8-5dec-4e3a-a238-580c77cb4810>], [<__main__.AtomicIncrementFlagTask with name 90b60a28-e5bd-470c-978e-d695fe141277>, <__main__.AtomicIncrementFlagTask with name 60c8daee-8573-4495-8ec2-9479e9c74499>], [<__main__.AtomicDecrementFlagTask with name 774d07dd-6069-47d4-ac0a-a6b6b26f02d7>, <__main__.AtomicDecrementFlagTask with name edb5faa0-d2ff-4faf-90da-6a57dfa174cf>, <__main__.AtomicDecrementFlagTask with name 0d6f8da6-05d1-4828-888d-242e2f833d15>, <__main__.AtomicDecrementFlagTask with name 2f73ecb6-a04d-42d5-8ad0-d35f5c86ac12>, <__main__.AtomicDecrementFlagTask with name a26d2da9-0b0f-4573-97fe-c33cbf243358>, <__main__.AtomicDecrementFlagTask with name eb5f2fef-eb47-48a1-9008-424956b9c5ab>, <__main__.AtomicDecrementFlagTask with name f12bff99-10ec-42ff-b984-8dfdcf9baed5>, <__main__.AtomicDecrementFlagTask with name f0a761c5-0e65-4fbb-a448-20f04950dd52>], [<__main__.GetFlagTask with name 5b0fd7fe-c9

In [5]:
client = RemoteClient(endpoint="http://127.0.0.1:26611", login="test", password="test")
client.healthcheck()

True

In [6]:
nodes = client.get_nodes()
for node in nodes:
    print(node)

current_nodes = nodes.take(1)

[dockerhost]


In [7]:
experiment = Experiment().map(pipeline, current_nodes)
print(experiment)

<Deployment: Node=dockerhost, executor_id=, prepared=False>


In [8]:
try:
    client.delete_experiment(experiment_name)
except Exception:
    pass

client.prepare_experiment(experiment, experiment_name)

'my_cool_experiment'

In [9]:
client.get_experiment_status(experiment_name)

ExperimentExecutionInformation(status=<ExperimentStatus.READY: 2>, experiment=<Deployment: Node=dockerhost, executor_id=237bc1dd-989a-4b9e-8376-2dc5aba86480, prepared=True>, execution_result=None)

In [10]:
client.start_execution(experiment_name)

'my_cool_experiment'

In [11]:
client.get_experiment_status(experiment_name)

ExperimentExecutionInformation(status=<ExperimentStatus.FINISHED: 4>, experiment=<Deployment: Node=dockerhost, executor_id=237bc1dd-989a-4b9e-8376-2dc5aba86480, prepared=True>, execution_result=[DeploymentExecutionResult(node=dockerhost, result=[<Success: defaultdict(<class 'list'>, {'c27b05f8-5dec-4e3a-a238-580c77cb4810': [<Success: 0>], '90b60a28-e5bd-470c-978e-d695fe141277': [<Success: 0>], '60c8daee-8573-4495-8ec2-9479e9c74499': [<Success: 0>], '774d07dd-6069-47d4-ac0a-a6b6b26f02d7': [<Success: 0>], 'edb5faa0-d2ff-4faf-90da-6a57dfa174cf': [<Success: 0>], '0d6f8da6-05d1-4828-888d-242e2f833d15': [<Success: 0>], '2f73ecb6-a04d-42d5-8ad0-d35f5c86ac12': [<Success: 0>], 'a26d2da9-0b0f-4573-97fe-c33cbf243358': [<Success: 0>], 'eb5f2fef-eb47-48a1-9008-424956b9c5ab': [<Success: 0>], 'f12bff99-10ec-42ff-b984-8dfdcf9baed5': [<Success: 0>], 'f0a761c5-0e65-4fbb-a448-20f04950dd52': [<Success: 0>], '5b0fd7fe-c991-44d4-a167-5b4c8fbd59bb': [<Success: text_value=None int_value=-6>]})>, ['Parsed conf

In [12]:
client.get_flag_values(experiment_name, flag_name)

FlagValues(text_value=None, int_value=-6)